In [1]:
!wget https://raw.githubusercontent.com/wandb/edu/main/mlops-001/lesson1/requirements.txt
!wget https://raw.githubusercontent.com/wandb/edu/main/mlops-001/lesson1/utils.py

!pip install -r requirements.txt
import gdown
url = 'https://drive.google.com/file/d/1Ns80_G8fAA0xs4e-7yvCVToYD-D5bNmb/view?usp=sharing'
output = 'params.py'
gdown.download(url=url, output=output, quiet=False, fuzzy=True)

--2024-12-07 01:02:02--  https://raw.githubusercontent.com/wandb/edu/main/mlops-001/lesson1/requirements.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 82 [text/plain]
Saving to: ‘requirements.txt’

requirements.txt    100%[===================>]      82  --.-KB/s    in 0s      

2024-12-07 01:02:02 (1.32 MB/s) - ‘requirements.txt’ saved [82/82]

--2024-12-07 01:02:02--  https://raw.githubusercontent.com/wandb/edu/main/mlops-001/lesson1/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6721 (6.6K) [text/plain]
Saving to

Downloading...
From (original): https://drive.google.com/uc?id=1Ns80_G8fAA0xs4e-7yvCVToYD-D5bNmb
From (redirected): https://drive.google.com/uc?id=1Ns80_G8fAA0xs4e-7yvCVToYD-D5bNmb&confirm=t&uuid=b712845d-02a2-4f21-959a-9eb56e76e3f2
To: /content/params.py
100%|██████████| 278/278 [00:00<00:00, 273kB/s]


'params.py'

In [ ]:
import os, warnings
import wandb

import pandas as pd
from fastai.vision.all import *
from sklearn.model_selection import StratifiedGroupKFold

import params
warnings.filterwarnings('ignore')

In [ ]:
run = wandb.init(project=params.WANDB_PROJECT, entity=params.ENTITY, job_type="data_split")

raw_data_at = run.use_artifact(f'{params.RAW_DATA_AT}:latest')
path = Path(raw_data_at.download())
path.ls()

wandb:   1 of 1 files downloaded.  


(#1) [Path('/content/artifacts/diabetes:v4/eda_table.table.json')]

In [ ]:
orig_eda_table = raw_data_at.get('eda_table.table.json')
print(orig_eda_table.columns)
print(orig_eda_table.data)

wandb:   1 of 1 files downloaded.  


['id', 'preg', 'plas', 'pres', 'skin', 'insu', 'mass', 'pedi', 'age', 'classe']
[['sample_0', 6, 148, 72, 35, 0, 33.6, 0.627, 50, 'tested_positive'], ['sample_1', 1, 85, 66, 29, 0, 26.6, 0.351, 31, 'tested_negative'], ['sample_2', 8, 183, 64, 0, 0, 23.3, 0.672, 32, 'tested_positive'], ['sample_3', 1, 89, 66, 23, 94, 28.1, 0.167, 21, 'tested_negative'], ['sample_4', 0, 137, 40, 35, 168, 43.1, 2.288, 33, 'tested_positive'], ['sample_5', 5, 116, 74, 0, 0, 25.6, 0.201, 30, 'tested_negative'], ['sample_6', 3, 78, 50, 32, 88, 31.0, 0.248, 26, 'tested_positive'], ['sample_7', 10, 115, 0, 0, 0, 35.3, 0.134, 29, 'tested_negative'], ['sample_8', 2, 197, 70, 45, 543, 30.5, 0.158, 53, 'tested_positive'], ['sample_9', 8, 125, 96, 0, 0, 0.0, 0.232, 54, 'tested_positive'], ['sample_10', 4, 110, 92, 0, 0, 37.6, 0.191, 30, 'tested_negative'], ['sample_11', 10, 168, 74, 0, 0, 38.0, 0.537, 34, 'tested_positive'], ['sample_12', 10, 139, 80, 0, 0, 27.1, 1.441, 57, 'tested_negative'], ['sample_13', 1, 189, 

In [ ]:
from sklearn.model_selection import StratifiedKFold
import pandas as pd

# Converter o wandb.Table para um pandas DataFrame mantendo as colunas originais
data = pd.DataFrame(orig_eda_table.data)
print(data)
# Amostra estratificada
tamanho_amostra = 100
classes = data[9].unique()
print(classes)
qtde_por_classe = round(tamanho_amostra * len(classes))
amostra_por_classe = []

for c in classes:
    selecao_classe_atual = data.loc[data[9] == c]
    amostra_c = selecao_classe_atual.sample(n=qtde_por_classe)
    amostra_por_classe.append(amostra_c)

amostra_estratificada = pd.concat(amostra_por_classe)
print(amostra_estratificada)
amostra_estratificada.info()

              0   1    2   3   4    5     6      7   8                9
0      sample_0   6  148  72  35    0  33.6  0.627  50  tested_positive
1      sample_1   1   85  66  29    0  26.6  0.351  31  tested_negative
2      sample_2   8  183  64   0    0  23.3  0.672  32  tested_positive
3      sample_3   1   89  66  23   94  28.1  0.167  21  tested_negative
4      sample_4   0  137  40  35  168  43.1  2.288  33  tested_positive
..          ...  ..  ...  ..  ..  ...   ...    ...  ..              ...
763  sample_763  10  101  76  48  180  32.9  0.171  63  tested_negative
764  sample_764   2  122  70  27    0  36.8  0.340  27  tested_negative
765  sample_765   5  121  72  23  112  26.2  0.245  30  tested_negative
766  sample_766   1  126  60   0    0  30.1  0.349  47  tested_positive
767  sample_767   1   93  70  31    0  30.4  0.315  23  tested_negative

[768 rows x 10 columns]
['tested_positive' 'tested_negative']
              0   1    2   3   4    5     6      7   8                9
3

In [ ]:
from sklearn.model_selection import train_test_split

# Selecionando todos os atributos com exceção da classe
X = amostra_estratificada.iloc[:, :-1]  # Features
y = amostra_estratificada[9]  # Target variable (classe)

# Dividir em train (70%) e temp (30%)
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=42
)

# Dividir temp em valid (15%) e test (15%)
X_valid, X_test, y_valid, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42
)

# Exibir as proporções
print(f"Train: {len(X_train)}, Valid: {len(X_valid)}, Test: {len(X_test)}")

# Adicionar a coluna 'Stage' e a classe
X_train['Stage'] = 'train'
X_train['classe'] = y_train  # Adiciona a coluna de classe

X_test['Stage'] = 'test'
X_test['classe'] = y_test  # Adiciona a coluna de classe

X_valid['Stage'] = 'valid'
X_valid['classe'] = y_valid  # Adiciona a coluna de classe

# Concatenar todos os DataFrames para formar o final
df = pd.concat([X_train, X_test, X_valid], ignore_index=True)

# Exibir a contagem de amostras em cada estágio
print(df['Stage'].value_counts())

# Salvar os DataFrames individuais
X_train.to_csv('train.csv', index=False)
X_test.to_csv('test.csv', index=False)
X_valid.to_csv('valid.csv', index=False)

# Salvar o resultado combinado no arquivo 'data_split.csv'
df.to_csv('data_split.csv', index=False)


Train: 280, Valid: 60, Test: 60
Stage
train    280
test      60
valid     60
Name: count, dtype: int64


In [ ]:
print(df)

              0   1    2    3   4    5     6      7   8  Stage  \
0    sample_603   7  150   78  29  126  35.2  0.692  54  train   
1    sample_555   7  124   70  33  215  25.5  0.161  37  train   
2    sample_691  13  158  114   0    0  42.3  0.257  44  train   
3    sample_725   4  112   78  40    0  39.4  0.236  38  train   
4    sample_417   4  144   82  32    0  38.5  0.554  37  train   
..          ...  ..  ...  ...  ..  ...   ...    ...  ..    ...   
395  sample_104   2   85   65   0    0  39.6  0.930  27  valid   
396  sample_446   1  100   72  12   70  25.3  0.658  28  valid   
397  sample_141   5  106   82  30    0  39.5  0.286  38  valid   
398  sample_260   3  191   68  15  130  30.9  0.299  34  valid   
399  sample_667  10  111   70  27    0  27.5  0.141  40  valid   

              classe  
0    tested_positive  
1    tested_negative  
2    tested_positive  
3    tested_negative  
4    tested_positive  
..               ...  
395  tested_negative  
396  tested_negative  


In [ ]:

df.columns = ['id', 'preg', 'plas', 'pres', 'skin', 'insu', 'mass', 'pedi', 'age', 'Stage', 'classe']
print(df.columns)
print(df)

Index(['id', 'preg', 'plas', 'pres', 'skin', 'insu', 'mass', 'pedi', 'age',
       'Stage', 'classe'],
      dtype='object')
             id  preg  plas  pres  skin  insu  mass   pedi  age  Stage  \
0    sample_603     7   150    78    29   126  35.2  0.692   54  train   
1    sample_555     7   124    70    33   215  25.5  0.161   37  train   
2    sample_691    13   158   114     0     0  42.3  0.257   44  train   
3    sample_725     4   112    78    40     0  39.4  0.236   38  train   
4    sample_417     4   144    82    32     0  38.5  0.554   37  train   
..          ...   ...   ...   ...   ...   ...   ...    ...  ...    ...   
395  sample_104     2    85    65     0     0  39.6  0.930   27  valid   
396  sample_446     1   100    72    12    70  25.3  0.658   28  valid   
397  sample_141     5   106    82    30     0  39.5  0.286   38  valid   
398  sample_260     3   191    68    15   130  30.9  0.299   34  valid   
399  sample_667    10   111    70    27     0  27.5  0.141   

In [ ]:
processed_data_at = wandb.Artifact(params.PROCESSED_DATA_AT, type="split_data")

processed_data_at.add_file('data_split.csv')
processed_data_at.add_dir(path)

# Ensure all column names are strings before creating the wandb.Table
df.columns = df.columns.astype(str)
data_split_table = wandb.Table(dataframe=df)

processed_data_at.add(data_split_table, "eda_table_data_split")

run.log_artifact(processed_data_at)
run.finish()


wandb: Adding directory to artifact (/content/artifacts/diabetes:v4)... Done. 0.0s
